<a href="https://colab.research.google.com/github/avivaviv283/MidiGeneratingProject/blob/master/midi_file_test_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 35.8 MB/s eta 0:00:00


In [12]:
import os
import glob
import pickle
import numpy
import mysql.connector
import mysql
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Bidirectional
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from google.colab import files


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:

def get_notes():
  notes = []
  cnx = mysql.connector.connect(user='root', password='admin', host='192.168.16.7', database='mididataset')
  cursor = cnx.cursor()

  # Execute a query
  cursor.execute('select file from midi_file where genreid = 2')

  # Fetch the results
  files = cursor.fetchall()
  #For every file in the MidiFiles folder
  for file in files:
   midi = converter.parse(file)

  print("Parsing %s" % file)

  notes_to_parse = None

  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))

# save the notes to a file
  with open('/content/drive/MyDrive/Colab Notebooks/notes_file', 'wb') as filepath:
      pickle.dump(notes, filepath)


  return notes

In [5]:
def prepare_sequences(notes,n_vocab):
  sequence_length = 100

      # get all pitch names
  pitchnames = sorted(set(item for item in notes))

  # create a dictionary to map pitches to integers
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  network_input = []
  network_output = []

      # create input sequences and the corresponding outputs
  for i in range(0, len(notes) - sequence_length, 1):
      sequence_in = notes[i:i + sequence_length]
      sequence_out = notes[i + sequence_length]
      network_input.append([note_to_int[char] for char in sequence_in])
      network_output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)

      # reshape the input into a format compatible with LSTM layers
  network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
      # normalize input
  network_input = network_input / float(n_vocab)

  network_output = np_utils.to_categorical(network_output)
  print(network_output)

  return (network_input,network_output)

In [6]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(Bidirectional(LSTM(512)))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [17]:
notes = get_notes()

# download the file to local machine
files.download("/content/drive/MyDrive/Colab Notebooks/notes_file")

DatabaseError: ignored

In [ ]:
notes = get_notes()
print(notes)
n_vocab = len(set(notes))
print(n_vocab)
network_input, network_output = prepare_sequences(notes,n_vocab)

model = create_network(network_input,n_vocab)

model.fit(network_input, network_output, epochs=100, batch_size=64)

Parsing /content/drive/MyDrive/Colab Notebooks/MidiFiles/ahead_on_our_way_piano.mid


FileNotFoundError: ignored

In [ ]:
model.save_weights('model_weights.hdf5')

In [ ]:
files.download('/content/model_weights.hdf5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!curl ipecho.net/plain

35.229.79.80